In [1]:
import requests
from bs4 import BeautifulSoup
import urllib.request 
import csv
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

In [2]:
try:
    df = pd.read_csv('character_names.csv', encoding='utf-8')
except UnicodeDecodeError:
    try:
        df = pd.read_csv('character_names.csv', encoding='ISO-8859-1')
    except UnicodeDecodeError:
        df = pd.read_csv('character_names.csv', encoding='cp1252')

df.head()

,Id,Character,Game,Description
0,1,Brimstone,Valorant,NaN
1,2,Viper,Valorant,NaN
2,3,Omen,Valorant,NaN
3,4,Killjoy,Valorant,NaN
4,5,Cypher,Valorant,NaN


In [3]:
df['base_query'] = df['Game'] + ' ' + df['Character']
df.head()

,Id,Character,Game,Description,base_query
0,1,Brimstone,Valorant,NaN,Valorant Brimstone
1,2,Viper,Valorant,NaN,Valorant Viper
2,3,Omen,Valorant,NaN,Valorant Omen
3,4,Killjoy,Valorant,NaN,Valorant Killjoy
4,5,Cypher,Valorant,NaN,Valorant Cypher


In [4]:
extras = ['fan art', 'art', 'wallpaper']
for _ in extras:
    df['base_query_' + _ ] = df['base_query'] + ' ' + _

df.head()

,Id,Character,Game,Description,base_query,base_query_fan art,base_query_art,base_query_wallpaper
0,1,Brimstone,Valorant,NaN,Valorant Brimstone,Valorant Brimstone fan art,Valorant Brimstone art,Valorant Brimstone wallpaper
1,2,Viper,Valorant,NaN,Valorant Viper,Valorant Viper fan art,Valorant Viper art,Valorant Viper wallpaper
2,3,Omen,Valorant,NaN,Valorant Omen,Valorant Omen fan art,Valorant Omen art,Valorant Omen wallpaper
3,4,Killjoy,Valorant,NaN,Valorant Killjoy,Valorant Killjoy fan art,Valorant Killjoy art,Valorant Killjoy wallpaper
4,5,Cypher,Valorant,NaN,Valorant Cypher,Valorant Cypher fan art,Valorant Cypher art,Valorant Cypher wallpaper


In [5]:
def create_query_list(row):
    return [row['base_query'], row['base_query_fan art'], row['base_query_art'], row['base_query_wallpaper']]

df['queries'] = df.apply(create_query_list, axis=1)

df.head()

,Id,Character,Game,Description,base_query,base_query_fan art,base_query_art,base_query_wallpaper,queries
0,1,Brimstone,Valorant,NaN,Valorant Brimstone,Valorant Brimstone fan art,Valorant Brimstone art,Valorant Brimstone wallpaper,"[Valorant Brimstone, Valorant Brimstone fan ar..."
1,2,Viper,Valorant,NaN,Valorant Viper,Valorant Viper fan art,Valorant Viper art,Valorant Viper wallpaper,"[Valorant Viper, Valorant Viper fan art, Valor..."
2,3,Omen,Valorant,NaN,Valorant Omen,Valorant Omen fan art,Valorant Omen art,Valorant Omen wallpaper,"[Valorant Omen, Valorant Omen fan art, Valoran..."
3,4,Killjoy,Valorant,NaN,Valorant Killjoy,Valorant Killjoy fan art,Valorant Killjoy art,Valorant Killjoy wallpaper,"[Valorant Killjoy, Valorant Killjoy fan art, V..."
4,5,Cypher,Valorant,NaN,Valorant Cypher,Valorant Cypher fan art,Valorant Cypher art,Valorant Cypher wallpaper,"[Valorant Cypher, Valorant Cypher fan art, Val..."


In [6]:
df = df.drop(['Description','base_query','base_query_fan art','base_query_art','base_query_wallpaper'], axis =1)
df.head()

,Id,Character,Game,queries
0,1,Brimstone,Valorant,"[Valorant Brimstone, Valorant Brimstone fan ar..."
1,2,Viper,Valorant,"[Valorant Viper, Valorant Viper fan art, Valor..."
2,3,Omen,Valorant,"[Valorant Omen, Valorant Omen fan art, Valoran..."
3,4,Killjoy,Valorant,"[Valorant Killjoy, Valorant Killjoy fan art, V..."
4,5,Cypher,Valorant,"[Valorant Cypher, Valorant Cypher fan art, Val..."


In [7]:
import requests
from bs4 import BeautifulSoup
from tqdm.auto import tqdm

# Function to get image URLs for a given search query
def get_image_urls(search_query):
    search_url = 'https://www.bing.com/images/search'
    params = {'q': search_query}
    response = requests.get(search_url, params=params)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        image_tags = soup.find_all('img', class_=lambda x: x and 'mimg' in x)
        # Extract 'src' or 'data-src' attribute based on which is available
        image_srcs = [img['src'] if img.has_attr('src') else img['data-src'] for img in image_tags if img.has_attr('src') or img.has_attr('data-src')]
        return image_srcs[:50]  # Return up to 50 image source URLs
    else:
        print(f'An error occurred while searching for: {search_query}')
        return []  # Return an empty list in case of an error

# Assuming df is your DataFrame and it already contains a 'queries' column with lists of queries
# We'll first define a new column 'urls' with empty lists
df['urls'] = pd.Series([[] for _ in range(len(df))], index=df.index)

# Iterate over the DataFrame rows with a progress bar
for index in tqdm(df.index, desc='Processing Characters'):
    all_image_urls = []
    row = df.loc[index]
    # For each query in the list of queries, perform a search and collect image URLs
    for query in row['queries']:
        image_urls = get_image_urls(query)
        all_image_urls.extend(image_urls)
        # Stop if we've collected 50 URLs, no need for further searches
        if len(all_image_urls) >= 50:
            break

    # Assign up to 50 URLs to the 'urls' column for this row
    df.at[index, 'urls'] = all_image_urls[:50]

# Display the DataFrame to verify the results
print(df)


Processing Characters:   0%|          | 0/646 [00:00<?, ?it/s]

      Id         Character       Game  \
0      1         Brimstone   Valorant   
1      2             Viper   Valorant   
2      3              Omen   Valorant   
3      4           Killjoy   Valorant   
4      5            Cypher   Valorant   
..   ...               ...        ...   
641  642         Kerillian  vermitide   
642  643  Victor Saltzpyre  vermitide   
643  644     Markus Kruber  vermitide   
644  645      Franz Lohner  vermitide   
645  646  Bardin Goreksson  vermitide   

                                               queries  \
0    [Valorant Brimstone, Valorant Brimstone fan ar...   
1    [Valorant Viper, Valorant Viper fan art, Valor...   
2    [Valorant Omen, Valorant Omen fan art, Valoran...   
3    [Valorant Killjoy, Valorant Killjoy fan art, V...   
4    [Valorant Cypher, Valorant Cypher fan art, Val...   
..                                                 ...   
641  [vermitide Kerillian, vermitide Kerillian fan ...   
642  [vermitide Victor Saltzpyre, vermitide

In [20]:
len(df['urls'][0])

50

In [21]:
df['urls']

0      [https://tse1.mm.bing.net/th/id/OIP.CCWpJMW-Bf...
1      [https://tse3.mm.bing.net/th/id/OIP.iGQ_iExsDi...
2      [https://tse4.mm.bing.net/th/id/OIP.Bprq17u0Tr...
3      [https://tse2.mm.bing.net/th/id/OIP.BI3cuduf1t...
4      [https://tse3.mm.bing.net/th/id/OIP.5oH3--pkmi...
                             ...                        
641    [https://tse1.mm.bing.net/th/id/OIP.9rIiSRWHqa...
642    [https://tse1.mm.bing.net/th/id/OIP.8owv6MhWeo...
643    [https://tse3.mm.bing.net/th/id/OIP.UDf9RHZ-63...
644    [https://tse3.mm.bing.net/th/id/OIP.X72JGsZj4S...
645    [https://tse3.mm.bing.net/th/id/OIP.hYH3sCOmN_...
Name: urls, Length: 646, dtype: object

In [22]:
import os
import requests
from tqdm.auto import tqdm

# Base directory where character folders will be created
base_dir = 'character_images'

# Ensure the base directory exists
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

# Function to download an image from a URL
def download_image(url, folder_path, image_number):
    try:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            # Create a valid filename from the image number
            filename = f'image_{image_number:03}.jpg'
            filepath = os.path.join(folder_path, filename)
            with open(filepath, 'wb') as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)
            return True
    except Exception as e:
        print(f"An error occurred: {e}")
    return False

# Iterate over the DataFrame rows and download images
for index, row in df.iterrows():
    character = row['Character']
    # Make a valid directory name for the character
    character_dir = os.path.join(base_dir, character.replace(' ', '_'))
    # Ensure the character's directory exists
    if not os.path.exists(character_dir):
        os.makedirs(character_dir)
    
    # Download each image from the 'urls' list
    for image_number, url in enumerate(tqdm(row['urls'], desc=f"Downloading images for {character}")):
        download_image(url, character_dir, image_number)


NotADirectoryError: [WinError 267] The directory name is invalid: 'character_images\\SOLDIER:_76'

In [24]:
resume_row_id = 216

for index, row in df.iterrows():
    if index >= resume_row_id:
        character = row['Character']
        # Replace characters that are invalid for directory names
        safe_character = ''.join(c for c in character if c.isalnum() or c in (' ','_')).rstrip()
        character_dir = os.path.join(base_dir, safe_character.replace(' ', '_'))
        
        # Ensure the character's directory exists
        if not os.path.exists(character_dir):
            os.makedirs(character_dir)
        
        # Get the list of images that have already been downloaded to avoid re-downloading
        existing_images = set(os.listdir(character_dir))
        
        # Download each image from the 'urls' list
        for image_number, url in enumerate(tqdm(row['urls'], desc=f"Downloading images for {character}")):
            # Check if the image already exists
            image_filename = f'image_{image_number:03}.jpg'
            if image_filename not in existing_images:
                if download_image(url, character_dir, image_number):
                    existing_images.add(image_filename) 